In [1]:
import numpy as np 
import pandas as pd
import hvplot.pandas
import hvplot as hv
import os 
import panel as pn
import matplotlib.pyplot as plt
pn.extension()
from secoora import get_bdat,get_mdat,fmt_ndbc_time,get_mod_idxs

In [2]:
def fmt_stamps(stamps):
    fmt = lambda stp : stp[4:6] + '/' + stp[6:8] + '/' + stp[:4] + ' ' + str(int(stp[8:])) + ':00'
    return list(map(fmt,np.array(stamps,dtype=str)))

In [3]:
def buoy_df(buoys,stamps,fields=['WDIR','WSPD','GST','WVHT','DPD','APD','MWD',
                         'PRES','ATMP','WTMP']):
    
    bdat,peaks,mins = get_bdat(stamps,buoys,fields)
    i=0
    df_dict = {'time':fmt_stamps(np.array(stamps,dtype=int))}
    for buoy in buoys:
        for field in fields:
            df_dict[buoy +' '+ field] = bdat[:,i]
            i+=1
    return pd.DataFrame(df_dict)

In [4]:
get_bdat,get_mdat

(<function secoora.interior_funcs.get_bdat(stamps, moorings, fields)>,
 <function secoora.interior_funcs.get_mdat(storm, bounds)>)

In [5]:
mod_data = np.load('sst.npy',allow_pickle=True)

In [6]:
bounds = (-70,23)
buoys = ['41013','41004']
# fields = ['WVHT','WSPD','DPD']
a=get_mdat('FLORENCE',bounds)
stamps = a[-1]

In [7]:
df = buoy_df(buoys,stamps)


C:\Users\twhes\Projects\secoora_\secoora\make_hr.py:35: RuntimeWarning: Mean of empty slice
  out[out_idx,1:] =  np.nanmean(data[idx:idx+step,:],axis=0)


In [8]:
df

,time,41013 WDIR,41013 WSPD,41013 GST,41013 WVHT,41013 DPD,41013 APD,41013 MWD,41013 PRES,41013 ATMP,...,41004 WDIR,41004 WSPD,41004 GST,41004 WVHT,41004 DPD,41004 APD,41004 MWD,41004 PRES,41004 ATMP,41004 WTMP
0,09/12/2018 12:00,64.333333,5.333333,6.500000,1.02,10.00,6.73,93.0,1018.300000,28.300000,...,81.000000,5.566667,6.816667,0.85,10.00,5.42,98.0,1017.350000,27.750000,29.166667
1,09/12/2018 13:00,66.833333,6.300000,7.566667,1.03,13.79,6.19,115.0,1018.500000,28.283333,...,68.333333,5.650000,7.016667,0.98,10.81,6.51,106.0,1017.766667,28.083333,29.116667
2,09/12/2018 14:00,63.333333,6.333333,7.683333,1.11,12.12,6.22,116.0,1018.916667,28.433333,...,69.833333,5.300000,6.583333,0.89,10.81,6.10,112.0,1018.200000,28.300000,29.100000
3,09/12/2018 15:00,54.000000,6.866667,8.283333,1.16,13.79,6.06,127.0,1018.600000,28.533333,...,71.833333,5.200000,7.316667,0.86,10.00,5.98,101.0,1018.316667,28.200000,29.150000
4,09/12/2018 16:00,51.666667,7.033333,8.766667,1.26,13.79,6.02,124.0,1018.316667,28.533333,...,71.833333,6.033333,7.516667,0.96,10.00,5.36,98.0,1017.883333,28.683333,29.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,09/18/2018 2:00,183.833333,6.133333,7.733333,1.30,6.25,4.96,162.0,1015.283333,27.816667,...,184.333333,5.483333,6.666667,1.13,9.09,5.12,126.0,1014.866667,27.900000,27.200000
135,09/18/2018 3:00,183.833333,6.700000,8.233333,1.31,10.00,4.99,121.0,1015.100000,27.816667,...,184.166667,6.250000,7.633333,1.24,10.81,5.13,100.0,1014.833333,27.966667,27.183333
136,09/18/2018 4:00,189.833333,6.850000,8.350000,1.25,5.56,4.96,151.0,1015.166667,27.816667,...,190.000000,6.733333,8.350000,1.29,12.12,5.14,104.0,1014.950000,28.000000,27.050000
137,09/18/2018 5:00,186.166667,7.033333,8.783333,1.20,8.33,5.00,110.0,1014.883333,27.816667,...,194.000000,6.350000,7.650000,1.22,12.12,4.93,116.0,1014.850000,28.016667,27.000000


In [9]:
def disp_buoys(dataframe,buoys,field,stamp):
    

    cols = [b+' '+field for b in buoys]
    
    points =dataframe[cols][dataframe['time']==stamp]

    return dataframe.hvplot(y=cols)*points.hvplot.scatter(y=cols)

In [10]:
hr_slider = pn.widgets.DiscreteSlider(value=list(df['time'])[0],options=list(df['time']))
dtype = pn.widgets.Select(name='Data Field',value = 'WVHT',options = ['WDIR','WSPD','GST','WVHT','DPD','APD','MWD',
                         'PRES','ATMP','WTMP'])
dash = pn.Row(hr_slider,dtype)
dash

Row
    [0] DiscreteSlider(options=['09/12/2018 12:00', ...], value='09/12/2018 12:00')
    [1] Select(name='Data Field', options=['WDIR', 'WSPD', ...], value='WVHT')

In [11]:
buoy_plot = pn.bind(disp_buoys, dataframe=df, buoys=buoys, field=dtype, stamp=hr_slider)

In [12]:
bapp = pn.Column(dash,buoy_plot)
bapp

Column
    [0] Row
        [0] DiscreteSlider(options=['09/12/2018 12:00', ...], value='09/12/2018 12:00')
        [1] Select(name='Data Field', options=['WDIR', 'WSPD', ...], value='WVHT')
    [1] ParamFunction(function, _pane=HoloViews, defer_load=False)

In [ ]:
np.array(df).shape

In [ ]:
buoy_box = pn.widgets.Select(name='Mooring',value = '41013',options = list(station_map.keys()))
dtype_box = pn.widgets.Select(name='Data Type',value = 'Sig. Wave Height',options =[lab_map[k] for k in lab_map.keys()])
event_box = pn.widgets.Select(name='Event',value = 'Florence',options = list(event_map.keys()))
  
dash = pn.bind(disp_buoy,dtype=dtype_box,station=buoy_box,event=event_box)


col = pn.Column(event_box,buoy_box,dtype_box,dash).servable(target='buoy_app')